# Comparison with article

In [20]:
from coupled_cluster.schemes.ccd_sparse import CoupledClusterDoublesSparse
from coupled_cluster.hartree_fock.scf_rhf import scf_rhf
from coupled_cluster.matrix_elements.generate_matrices import (
    get_one_body_elements, get_coulomb_elements,
    get_antisymmetrized_elements, add_spin_to_one_body_elements,
    get_one_body_elements_spin
)
from coupled_cluster.hartree_fock.basis_transformation import (
    transform_one_body_elements, transform_two_body_elements
)

import numpy as np
import time

In [21]:
omega = 0.5
l = 30
n = 2

print ("""
w = {0},
l = {1},
n = {2}
""".format(omega, l, n))

h = omega * get_one_body_elements(l)
u = np.sqrt(omega) * get_coulomb_elements(l)

c, energy = scf_rhf(h.todense(), u, np.eye(l//2), n//2)

print ("RHF Energy: {0}".format(energy))

hi = transform_one_body_elements(h, c)
oi = transform_two_body_elements(u, c)

_h = add_spin_to_one_body_elements(hi, l)
_u = get_antisymmetrized_elements(l, oi=oi)

ccd_hf = CoupledClusterDoublesSparse(_h, _u, n)
print ("CCD (HF) Energy: {0}".format(ccd_hf.compute_energy()))

__h = omega * get_one_body_elements_spin(l)
__u = np.sqrt(omega) * get_antisymmetrized_elements(l)

ccd = CoupledClusterDoublesSparse(__h, __u, n)
print ("CCD Energy: {0}".format(ccd.compute_energy()))


w = 0.5,
l = 30,
n = 2

RHF Energy: 1.7998563827038119
CCD (HF) Energy: 1.672689553578182
CCD Energy: 1.7590538638203967
